In [ ]:
from csidhtools import CSIDHCW
from csidhtools.search.io import *
import chipwhisperer as cw
from tqdm import tqdm, trange
import random

In [ ]:
if "csidh" in globals() or "csidh" in locals():
    csidh.reset_target()
    csidh.dis()

In [ ]:
csidh = CSIDHCW(PATH + "/csidh-target/src/")

In [ ]:
if "BENCH_MODE" in globals():
    csidh.BENCH_MODE = BENCH_MODE

if "ATTACK_TYPE" in globals():
    csidh.ATTACK_TYPE = ATTACK_TYPE

if "OPT" in globals():
    csidh.OPT = OPT

if "SCOPETYPE" in globals():
    csidh.SCOPETYPE = SCOPETYPE

if "SS_VER" in globals():
    csidh.SS_VER = SS_VER

if "PLATFORM" in globals():
    csidh.PLATFORM = PLATFORM

In [ ]:
csidh.setup()

In [ ]:
csidh.flash_target()
csidh.reset_target()

In [ ]:
csidh.scope.clock.clkgen_src = "system"
csidh.scope.clock.clkgen_freq = 14740259
csidh.target.baud = 460800
csidh.scope.clock.adc_mul = 1

Change frequency snippet

In [ ]:
# CLOCK = csidh.scope.clock.clkgen_freq
# BAUD = csidh.target.baud
# print('Clock: %f MHz\nBaud: %d bps' % (CLOCK, BAUD))
# NEWCLOCK = CLOCK * 2
# csidh.scope.clock.clkgen_freq = NEWCLOCK
# csidh.target.baud = BAUD  * (NEWCLOCK/CLOCK)
# print('Clock: %f MHz\nBaud: %d bps' % (csidh.scope.clock.clkgen_freq, csidh.target.baud))

Test run snippet

In [ ]:
# csidh.reset_target()
# csidh.scope.arm()
# csidh.action()
# ret = csidh.scope.capture()
# if ret:
#     print("Timeout happened during acquisition")
# PUBLIC_EXPECTED = csidh.public_with_errors
# MAX_EXT_OFFSET = csidh.scope.adc.trig_count
# print("#" * 80)
# print("First run of CSIDH")
# print("#" * 80)
# print(f"PUBLIC: {PUBLIC_EXPECTED}")
# print(f"TRIGS:  {MAX_EXT_OFFSET}")
# print("#" * 80)